In [1]:
using ExpandLGF
using OffsetArrays
include("../test/test_utils.jl");

Time the evaluation of near-field LGF entries for Mehrstellen stencils using a combination of quadratue and series expansion. 

Times are reported for the generation of far-field expansions via symbolic computation, the evalution of near-field values using numerical quadrature, and the evaluation of far-field values using a series expansion.

In [ ]:
###
# Parameters
###
N = 64            # Time evaluation on [0, N]^3
M = 128           # Test residual on [0, M]^3
near_terms = 10   # terms in near-field expansions
far_terms = 8     # terms in far-field expansion
rtol = 0          # relative tolerance on evaluations
atol = 1e-13      # absolute tolerance on evaluations
maxevals = 10^11  # maximum number of quadrature points for each evaluation

In [ ]:
function get_timings(name, stencil)

    ###
    # Time the generation of all the expansion objects
    ###
    expansion_time = @elapsed begin
        ELGF = ExpandLGF.EvaluateLGF(stencil, far_terms, rtol, atol, maxevals)
        nmax = ExpandLGF.near_cutoff(ELGF)
        NF = ExpandLGF.near_field(ELGF)
        FF = ExpandLGF.far_field(ELGF)
    end

    ###
    # Time the near and far field evaluations
    ###
    lgf = OffsetArray(zeros(Float64, (N, N, N)), 0:N-1, 0:N-1, 0:N-1)
    evaluation_indices = sorted_indices_3D(N-1)
    near_field_indices = collect(n for n in evaluation_indices if sum(n.^2) < nmax^2)
    far_field_indices = collect(n for n in evaluation_indices if sum(n.^2) >= nmax^2)

    println("Evaluating near-field for $(name)")
    println("$(stencil)")
    println("Total near-field evaluations: $(length(near_field_indices)) for points with |n| < $(nmax)")
    println("Total far-field evaluations: $(length(far_field_indices)) for points with |n| ≥ $(nmax)")

    near_field_time = @elapsed for index in near_field_indices
        lgf[index...] = NF(index)
    end

    far_field_time = @elapsed for index in far_field_indices
        lgf[index...] = FF(index)
    end

    println("$(expansion_time)s for expansion setup")
    println("$(near_field_time)s for near field")
    println("$(far_field_time)s for far field")

    ###
    # Build a larger block of size M^3
    ###
    filled_block = generate_full_test_block(ExpandLGF.left_stencil(stencil), lgf)
    test_block = OffsetArray(zeros(Float64, (M + 1, M + 1, M + 1)), 0:M, 0:M, 0:M)
    test_block[0:N-1, 0:N-1, 0:N-1] = filled_block[0:N-1, 0:N-1, 0:N-1]

    FF_flups = ExpandLGF.FarField(stencil, 4)
    for n in CartesianIndices(test_block)
        n = Tuple(n)
        if maximum(n) >= N
            test_block[n...] = FF_flups([n...])
        end
    end

    ###
    # Evaluate the max residual on the larger block
    ###
    residual = apply_stencil(ExpandLGF.left_stencil(stencil), test_block)
    handle_origin!(residual)
    max_residual, index = findmax(abs, residual)
    @printf("Max residual in test block of size [0, %d]^3: %1.3e", M, max_residual)
    print(" at index $(Tuple(index))\n\n")
end

In [ ]:
###
# Print timing results for each Mehrstellen stencil
###
names = "MEH" .* string.([4, 6])
stencils = [ExpandLGF.Mehrstellen4(), ExpandLGF.Mehrstellen6()]

# precompile code used for series expansions
ELGF = ExpandLGF.EvaluateLGF(stencils[1], far_terms, rtol, atol, maxevals)

for (name, stencil) in zip(names, stencils)
    get_timings(name, stencil)
end

Full output (on a 2021 MacBook Pro with Apple M1 Pro processor)
```
Evaluating near-field for MEH4
ExpandLGF.MehrstellenStencil{3, Rational{Int64}}(ExpandLGF.FullStencil{3, Rational{Int64}}(1, [(0, -1, -1), (-1, 0, -1), (0, 0, -1), (1, 0, -1), (0, 1, -1), (-1, -1, 0), (0, -1, 0), (1, -1, 0), (-1, 0, 0), (0, 0, 0), (1, 0, 0), (-1, 1, 0), (0, 1, 0), (1, 1, 0), (0, -1, 1), (-1, 0, 1), (0, 0, 1), (1, 0, 1), (0, 1, 1)], Rational{Int64}[1//6, 1//6, 1//3, 1//6, 1//6, 1//6, 1//3, 1//6, 1//3, -4//1, 1//3, 1//6, 1//3, 1//6, 1//6, 1//6, 1//3, 1//6, 1//6]), ExpandLGF.FullStencil{3, Rational{Int64}}(1, [(0, 0, -1), (0, -1, 0), (-1, 0, 0), (0, 0, 0), (1, 0, 0), (0, 1, 0), (0, 0, 1)], Rational{Int64}[1//12, 1//12, 1//12, 1//2, 1//12, 1//12, 1//12]))
Total near-field evaluations: 141 for points with |n| < 10
Total far-field evaluations: 45619 for points with |n| ≥ 10
11.665535958s for expansion setup
9412.896692792s for near field
1.542155833s for far field
Max residual in test block of size [0, 128]^3: 1.969e-12 at index (6, 6, 6)

Evaluating near-field for MEH6
ExpandLGF.MehrstellenStencil{3, Rational{Int64}}(ExpandLGF.FullStencil{3, Rational{Int64}}(1, [(-1, -1, -1), (0, -1, -1), (1, -1, -1), (-1, 0, -1), (0, 0, -1), (1, 0, -1), (-1, 1, -1), (0, 1, -1), (1, 1, -1), (-1, -1, 0), (0, -1, 0), (1, -1, 0), (-1, 0, 0), (0, 0, 0), (1, 0, 0), (-1, 1, 0), (0, 1, 0), (1, 1, 0), (-1, -1, 1), (0, -1, 1), (1, -1, 1), (-1, 0, 1), (0, 0, 1), (1, 0, 1), (-1, 1, 1), (0, 1, 1), (1, 1, 1)], Rational{Int64}[1//30, 1//10, 1//30, 1//10, 7//15, 1//10, 1//30, 1//10, 1//30, 1//10, 7//15, 1//10, 7//15, -64//15, 7//15, 1//10, 7//15, 1//10, 1//30, 1//10, 1//30, 1//10, 7//15, 1//10, 1//30, 1//10, 1//30]), ExpandLGF.FullStencil{3, Rational{Int64}}(2, [(0, 0, -2), (0, -1, -1), (-1, 0, -1), (0, 0, -1), (1, 0, -1), (0, 1, -1), (0, -2, 0), (-1, -1, 0), (0, -1, 0), (1, -1, 0), (-2, 0, 0), (-1, 0, 0), (0, 0, 0), (1, 0, 0), (2, 0, 0), (-1, 1, 0), (0, 1, 0), (1, 1, 0), (0, 2, 0), (0, -1, 1), (-1, 0, 1), (0, 0, 1), (1, 0, 1), (0, 1, 1), (0, 0, 2)], Rational{Int64}[-1//240, 1//90, 1//90, 1//18, 1//90, 1//90, -1//240, 1//90, 1//18, 1//90, -1//240, 1//18, 67//120, 1//18, -1//240, 1//90, 1//18, 1//90, -1//240, 1//90, 1//90, 1//18, 1//90, 1//90, -1//240]))
Total near-field evaluations: 106 for points with |n| < 9
Total far-field evaluations: 45654 for points with |n| ≥ 9
13.852254375s for expansion setup
6264.964332s for near field
1.483771542s for far field
Max residual in test block of size [0, 128]^3: 8.090e-13 at index (4, 1, 8)
```
